# Diaspora Event SDK - v3 API Example

This notebook demonstrates all v3 APIs in a logical order:
1. User Management (create_user, delete_user)
2. Key Management (create_key, get_key, delete_key)
3. Namespace Management (list_namespaces)
4. Topic Management (create_topic, recreate_topic, delete_topic)
5. Kafka Operations (produce and consume messages)


In [ ]:
# Setup
# Install dependencies and initialize the client.

%pip install -e '.[kafka-python]'

import os
import uuid  # noqa: E402
import json  # noqa: E402
from datetime import datetime  # noqa: E402
from diaspora_event_sdk import Client as GlobusClient  # noqa: E402
from diaspora_event_sdk.sdk.kafka_client import KafkaProducer, KafkaConsumer  # noqa: E402

# os.environ["DIASPORA_SDK_ENVIRONMENT"] = "local"
c = GlobusClient()

In [ ]:
# Display the current user's subject ID
print(f"Current user subject: {c.subject_openid}")
c.subject_openid

In [ ]:
# Step 1: User Management
# Create an IAM user with policy and namespace
# This is typically done once per user and creates a default namespace automatically

print("Creating IAM user (creates user, policy, and namespace)...")
user_result = c.create_user()
print(f"Status: {user_result.get('status', 'N/A')}")
print(f"Message: {user_result.get('message', 'N/A')}")
print(f"User created: {user_result.get('user_created', False)}")
print(f"Policy created: {user_result.get('policy_created', False)}")
print(f"Policy attached: {user_result.get('policy_attached', False)}")
print(f"Namespace created: {user_result.get('namespace_created', False)}")
print(f"Namespace: {user_result.get('namespace', 'N/A')}")

# Store the namespace for later use
namespace = user_result.get(
    "namespace", f"ns-{c.subject_openid.replace('-', '')[-12:]}"
)
print(f"\nUsing namespace: {namespace}")

In [ ]:
# Step 2: List Namespaces
# View all namespaces and their associated topics
print("Listing all namespaces and topics...")
namespaces_result = c.list_namespaces()
print(f"Status: {namespaces_result.get('status', 'N/A')}")
print(f"Message: {namespaces_result.get('message', 'N/A')}")

namespaces = namespaces_result.get("namespaces", {})
if namespaces:
    print("\nNamespaces and topics:")
    for ns, topics in namespaces.items():
        print(f"  {ns}: {topics}")
    # Use the first namespace if we don't have one from create_user

In [ ]:
# Step 3: Key Management
# Get or create access keys for Kafka operations

print("Getting access key (retrieves from DynamoDB if exists, creates new if not)...")
key_result = c.create_key()
print(f"Access Key: {key_result.get('access_key', 'N/A')[:20]}...")
print(f"Retrieved from DynamoDB: {key_result.get('retrieved_from_dynamodb', False)}")
print(f"Endpoint: {key_result.get('endpoint', 'N/A')}")

In [ ]:
# Step 4: Create Topic
# Create a topic under the namespace

topic_name = f"topic-{str(uuid.uuid4())[:5]}"
print(f"Creating topic '{topic_name}' under namespace '{namespace}'...")
topic_result = c.create_topic(namespace, topic_name)
print(f"Status: {topic_result.get('status', 'N/A')}")
print(f"Message: {topic_result.get('message', 'N/A')}")
print(f"Namespace: {topic_result.get('namespace', 'N/A')}")
print(f"Topic: {topic_result.get('topic', 'N/A')}")

# Kafka topic name format: "{namespace}.{topic}"
kafka_topic = f"{namespace}.{topic_name}"
print(f"\nKafka topic name: {kafka_topic}")

In [ ]:
# Step 5: Produce Messages
# Produce multiple messages to the topic using KafkaProducer

print(f"Producing messages to topic: {kafka_topic}")
print("Note: Wait a few seconds after creating the key for IAM policy to propagate...")
# time.sleep(5)  # Wait for IAM policy propagation

p = KafkaProducer(kafka_topic)
for i in range(3):
    message = {
        "message_id": i + 1,
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "content": f"Message {i + 1} from v3 API",
    }
    future = p.send(kafka_topic, message)
    result = future.get(timeout=30)
    print(f"Produced message {i + 1}: offset={result.offset}")
p.close()

In [ ]:
# Step 6: Consume Messages
# Consume messages from the topic using KafkaConsumer

print(f"\nConsuming messages from topic: {kafka_topic}")
consumer = KafkaConsumer(kafka_topic, auto_offset_reset="earliest")
messages = consumer.poll(timeout_ms=10000)
for tp, msgs in messages.items():
    for message in msgs:
        data = json.loads(message.value.decode("utf-8"))
        print(f"  Consumed: {data}")
consumer.close()

In [ ]:
# Step 7: Recreate Topic (Buggy)
# Recreate a topic by deleting and recreating it via KafkaAdminClient
# This is useful for resetting topic state or configuration

# print(f"Recreating topic '{topic_name}' under namespace '{namespace}'...")
# recreate_result = c.recreate_topic(namespace, topic_name)
# print(f"Status: {recreate_result.get('status', 'N/A')}")
# print(f"Message: {recreate_result.get('message', 'N/A')}")
# print(f"Namespace: {recreate_result.get('namespace', 'N/A')}")
# print(f"Topic: {recreate_result.get('topic', 'N/A')}")

In [ ]:
# Step 8: Delete Topic
# Delete the topic we created

print(f"Deleting topic '{topic_name}' from namespace '{namespace}'...")
delete_result = c.delete_topic(namespace, topic_name)
print(f"Status: {delete_result.get('status', 'N/A')}")
print(f"Message: {delete_result.get('message', 'N/A')}")
print(f"Namespace: {delete_result.get('namespace', 'N/A')}")
print(f"Topic: {delete_result.get('topic', 'N/A')}")

In [ ]:
# Step 9: Verify Deletion
# List namespaces again to confirm topic deletion

print("Verifying topic deletion...")
result = c.list_namespaces()
namespaces = result.get("namespaces", {})
if namespaces:
    print("Remaining namespaces and topics:")
    for ns, topics in namespaces.items():
        print(f"  {ns}: {topics}")
    # Check if our topic was deleted
    if namespace in namespaces:
        if topic_name not in namespaces[namespace]:
            print(
                f"\n✓ Topic '{topic_name}' successfully deleted from namespace '{namespace}'"
            )
        else:
            print(f"\n✗ Topic '{topic_name}' still exists in namespace '{namespace}'")
else:
    print("No namespaces found.")

In [ ]:
# Step 10: Optional Cleanup
# Delete access keys and user (use with caution - this deletes all user resources)

# Uncomment to delete access keys
# print("Deleting access keys...")
# delete_key_result = c.delete_key()
# print(f"Status: {delete_key_result.get('status', 'N/A')}")
# print(f"Message: {delete_key_result.get('message', 'N/A')}")

# Uncomment to delete user (deletes IAM user and all associated resources)
# print("\nDeleting user (deletes IAM user, policy, namespaces, and keys)...")
# delete_user_result = c.delete_user()
# print(f"Status: {delete_user_result.get('status', 'N/A')}")
# print(f"Message: {delete_user_result.get('message', 'N/A')}")
# print(f"Namespaces deleted: {delete_user_result.get('namespaces_deleted', False)}")
# print(f"Keys deleted: {delete_user_result.get('keys_deleted', False)}")
# print(f"Policy detached: {delete_user_result.get('policy_detached', False)}")
# print(f"Policy deleted: {delete_user_result.get('policy_deleted', False)}")
# print(f"User deleted: {delete_user_result.get('user_deleted', False)}")
